**1. Смотрим на Hadoop Distributed File System**

В рамках этой части вам нужно будет обращаться к HDFS с помощью CLI, разместить файлы для следующих заданий в распределеннй файловой системе и выполнить несколько преобразований над ними.

Для работы файлы можно скачать по следующим ссылкам:
- Логи посещения сайтов юзерами за некоторый промежуток времени [ссылка](https://drive.google.com/file/d/1WXyq5WVSWwJYXPuH4kyAJ5mrR3XgfO_H/view?usp=sharing)

Разместите их в нашем внутреннем файловом хранилище с помощью HDFS CLI, для дальнейшего удобства под каждый файл стоит создать каталог с простым и понятным именем, разместить сами файлы в разных каталогах.

Набор комманд, которые вам могут в этом помочь, доступны [здесь](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/FileSystemShell.html)

В ячейках ниже должен быть полный набор комманд ваших обращей к консоли

In [ ]:
## вы можете обращаться к консоли из ноутбука таким способом
!hdfs dfs -ls /

Found 3 items
drwxr-xr-x   - root supergroup          0 2024-06-01 07:40 /input
drwxr-xr-x   - root supergroup          0 2024-06-01 07:42 /output
drwxr-xr-x   - root supergroup          0 2024-06-01 07:34 /tmp


In [ ]:
%%bash
## или же использовать для этого меджик строчку в ячейке %%bash, как вам будет удобнее

hdfs dfs -ls /

Found 3 items
drwxr-xr-x   - root supergroup          0 2024-06-01 07:40 /input
drwxr-xr-x   - root supergroup          0 2024-06-01 07:42 /output
drwxr-xr-x   - root supergroup          0 2024-06-01 07:34 /tmp


In [ ]:
!hdfs dfs -ls -R -h /input

-rw-r--r--   1 root supergroup         24 2024-06-01 07:40 /input/input.txt


In [ ]:
## ваше решение здесь

**2. Решаем задачи MapReduce**

**2.1 Подсчет слов в тексте**

В рамках данного задания вам нужно подсчитать кол-во слов в тексте Библии (файл приложен к ДЗ в чате тг), то есть необходимо реализовать базовый функционал утилиты word count.

**Важно** - подсчитывайте число только тех слов, длина которых больше 4 символов. Проводить процесс удаления знаков препинания и прочих символов **не нужно**

Ниже вам представлены ячейки, в которых вы должны описать структуру маппера/редьсюера и ниже вызвать их в bash-скрипте запуска MR-таски

In [ ]:
%%writefile mapper.py
## сюда вы пишете питоновский скрипт для работы маппера


In [ ]:
%%writefile reducer.py
## сюда вы пишете питоновский скрипт для работы редьюсера


В качестве проверки ваших python-скриптов до запуска MR таски можно произвести их запуск через консольные команды

Тогда наша задача не будет выполняться через датаноды, а только на локальной машине, но в случае ошибок в скриптах вы увидите их и сможете исправить

In [ ]:
%%bash
## пример запуска скриптов на неймноде для проверки их работы

cat file | python3 mapper.py | sort -k1,1 (с нужными ключами сортировки) | python3 reducer.py

Как только в данной проверке вы получите успешный и корректный результат, можете запустить Map Reduce в ячейке ниже

In [ ]:
%%bash
## шаблон для запуска MR таски

# обязательная чистка директории, куда будем складывать результат отрабоки mr
hdfs dfs -rm -r /word_count_task || true

# запус mr таски с указанием пути до нужного jar
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="word-count" \
    -files ./mapper.py,./reducer.py \
    -mapper "python3 mapper.py" \
    -reducer "python3 reducer.py" \
    -input ## тут указать путь до нужного файла в hdfs
    -output /word_count_task

Мониторить процесс работы таски можно на nodemanager по порту 8088 (уже прокинут в конфиге), там будет UI, в котором будет видно вашу запущенную задачу и её статус.

Результат работы скрипта должен выглядеть следующим образом (вывод тестовый):

```bash
word count
abtr 6852
btoad 4237
stress 1932
zen 1885
```

In [ ]:
%%bash
## запустите эту команду, чтобы вывести счетчик определенных слов, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер

hdfs dfs -cat /word_count_task/* | grep  -E 'lord\.|god\.|pray\.' | sort -t$'\t' -k2.2nr  | head -n 3

**2.2 Решаем задачу поиска самых посещаемых сайтов**

В данном задании нужно поработать с логом данных о посещении юзерами различных сайтов.
Формат данных: `url;временная метка`. Вам нужно вывести топ 5 сайтов по посещаемости в каждую из дат, которая представлена в наших данных.

Результат работы скрипта должен выглядеть следующим образом:

```bash
date        site                            count
2024-05-25  https://gonzales-bautista.com/  987
2024-05-25  https://smith.com/              654
2024-05-25  https://www.smith.com/          321
```

**Рекомендации**

1. Вам могу пригодиться дополнительные параметры mr таски, отвечающие за настройку шаффла, и правил сортировки ключей внутри него. Почитать о примерах их использования можно [здесь](https://hadoop.apache.org/docs/current/hadoop-streaming/HadoopStreaming.html#More_Usage_Examples).

2. Не рекомендуем использовать `\t` в качестве символа разделителя для сложного ключа (потому что по дефолту таб используется для разделения колонок данных, и ключом в таком случае будет только первая колонка до таба). Если вы будете собирать сложный ключ для нужной вам сортировки данных, лучше всего будет использовать другие симловы, к примеру `+, =`.

3. Возможно, у вас не получится решить данную задачу за одну mr таску, тогда вы просто описываете в решении скрипты ваших мапперов, редьюсеров под каждую из mr тасок, которые вам нужно запустить для получения нужного результата.

**Важно** помнить, что любой маппер и редьюсер должен работать за O(1) памяти, и если вы будете создавать какой-то список, куда будете складывать какие-то данные, то он не должен быть размера O(n). Если такой момент в вашем решении будет, пожалуйста, поясните его текстово, что с вашими переменными все ок, и у них нет размера O(n).

In [ ]:
## ваше решение здесь

In [ ]:
%%bash
## запустите эту команду, чтобы вывести результат работы по определенным компаниям, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер
## укажите путь до той директории на hdfs, куда вы складывали результат

!hdfs dfs -cat /output/* | grep -E '2024-05-28|2024-06-02|2024-05-30' | column -t -s$'\t'